## n-Bit Quantum Ripple Adder
#### Assigmnet: Exam 1 Part 2
#### Class: EE 5885 Introduction to Quantum Computing
#### Instructor: Dr. Suresh
#### Author: Josh Blaney
#### Date: 10/27/2022

In [ ]:
#Import QuantumRegister and ClassicalRegister classes from qiskit library
from qiskit import QuantumRegister as QuR
from qiskit import ClassicalRegister as CR

#Import QuantumCircuit class, execute function, and noisy simulator Aer
from qiskit import QuantumCircuit as QuC
from qiskit import execute, Aer

import matplotlib
from qiskit.visualization import plot_histogram

In [ ]:
#Create a composite gate for the carry Function
qcarry = QuR(4)
cqc = QuC(qcarry, name='carry')
cqc.ccx(qcarry[1],qcarry[2],qcarry[3])
cqc.cx(qcarry[1],qcarry[2])
cqc.ccx(qcarry[0],qcarry[2],qcarry[3])

#Convert to a composite gate
carry_gate = cqc.to_instruction()

In [ ]:
# Create a composite gate for carry dagger
iqcarry = QuR(4)
icqc = QuC(iqcarry, name='inv carry')
icqc.ccx(iqcarry[0],iqcarry[2],iqcarry[3])
icqc.cx(iqcarry[1],iqcarry[2])
icqc.ccx(iqcarry[1],iqcarry[2],iqcarry[3])

# Convert to a composite gate
inv_carry_gate = icqc.to_instruction()

In [ ]:
#Create a composite gate for the sum function
qsum = QuR(3)
sqc = QuC(qsum, name='sum')
sqc.cx(qsum[1],qsum[2])
sqc.cx(qsum[0],qsum[2])

#Convert to a composite gate
sum_gate = sqc.to_instruction()

In [ ]:
# Function to implement base 10 to binary conversion with padding
def int_to_bits(w, bits):
    x = int(w)
    y = ""
    while (x > 0):
        r = x % 2
        y += str(r)
        x = x//2
    
    for i in range(len(y)-bits):
        y += "0"
    
    return y

In [ ]:
#Function to implement n-bit Ripple Adder
def nBitRippleAdder(Input1,Input2,CarryIn, n):
    Input1_bits = int_to_bits(Input1, n) # Find the binary equivalent of the input integer A
    Input2_bits = int_to_bits(Input2, n) # Find the binary equivalent of the input integer B
    q = QuR(3*n+1,'q') # Setup a quantum register with enough bits for the quantum circuit
    c = CR(3*n+1,'c')  # Setup a classical register with enough bits to store the result
    qc = QuC(q,c)      # Setup a quantum circuit pulling the two registers together
    
    #Setup Carry in Input
    if (CarryIn == 1):
        qc.x(q[0])
    
    # Iterate over the binary A and add X gates to set the input qubits accordingly
    for index, bit in enumerate(Input1_bits):
        if bit == '1':
            qc.x(q[(index*3)+1])
            
    # Iterate over the binary B and add X gates to set the input qubits accordingly
    for index, bit in enumerate(Input2_bits):
        if bit == '1':
            qc.x(q[(index*3)+2])
    
    # Record the compute qubits for the uncompute step
    uncompute = []
    
    # Now perform the compute portion of the circuit
    for i in range(0,n):
        j = 3 * i
        qc.append(carry_gate,[q[j],q[j+1],q[j+2],q[j+3]])
        uncompute.append([q[j],q[j+1],q[j+2],q[j+3]])
    
    qc.barrier()
    
    # Uncompute the A xor B in the final carry
    qc.cx(q[-3],q[-2])
    
    # Add the first sum gate
    # n sum gates are required but
    # n-1 inverse carry gates are required
    qc.append(sum_gate, [q[-4],q[-3],q[-2]])
    
    # Finish uncompute with n-1 inverse carry and sum gates
    for i in range(2,n+1):
        qc.append(inv_carry_gate, uncompute[-i])
        qc.append(sum_gate, uncompute[-i][0:3])
    
    qc.barrier()
    qc.measure(q,c)
    
    return qc

In [ ]:
in1 = 255 # Input A
in2 = 255 # Input B
cin = 1 # Carry In
bits = 8 # number of bits in the adder

QC = nBitRippleAdder(in1,in2,cin,n=bits)
QC.draw(output = 'mpl', scale = 2)

In [ ]:
#Create a simulator to perform noisy simulation using the qasm_simulator 
sim = Aer.get_backend('qasm_simulator')
#Perform the noisy simulation using the qasm_Simulator and repeat 1024 times
job = execute(QC,backend=sim,shots=1024)
#Obtain the results of the job submitted
result = job.result()
#Obtain the results of the simulation
counts= result.get_counts(QC)
print(counts)

In [ ]:
#Plot the output as a histogram
plot_histogram(counts)